### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [1]:
import os
from dotenv import load_dotenv # .env 파일 로드를 위해 추가
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# from langchain.docstore.document import Document # PDF 로드 시 필요 없음

# --- 0단계: 환경 변수 설정 ---
# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# OpenAI API 키를 환경 변수에서 가져옵니다.
# .env 파일에 OPENAI_API_KEY=YOUR_KEY 형태로 저장되어 있어야 합니다.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [2]:
# --- PDF 파일 로드 ---
pdf_file_path = "../data/콘텐츠분쟁해결_사례.pdf" # 실제 PDF 파일 경로를 확인해주세요.

try:
    loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()
    print(f"'{pdf_file_path}'에서 {len(documents)} 페이지의 문서 로드 완료.")
except Exception as e:
    print(f"PDF 파일을 로드하는 데 실패했습니다: {e}")
    print("PDF 파일이 './data/' 경로에 있는지, 파일명이 정확한지 확인해주세요.")
    # 파일 로드 실패 시 이 시점에서 스크립트 진행을 멈추는 것이 좋습니다.
    # 만약 예외 처리 후에도 계속 진행해야 한다면, 빈 리스트를 할당하여 다음 단계에서 오류를 방지할 수 있습니다.
    documents = [] # 오류 발생 시 빈 문서 리스트로 설정
    print("문서 로드에 실패하여 다음 단계로 진행할 수 없습니다. 파일을 확인해주세요.")


# --- 1단계: 문서 분할 설정 ---
# 문서가 성공적으로 로드되었을 때만 분할을 진행합니다.
if documents:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,  # 법률 사례는 1200-1800자 권장
        chunk_overlap=300,  # 사례 맥락 보존을 위해 200-400자
        separators=[
            "\n【사건개요】",  # 법률 문서 섹션 구분자
            "\n【쟁점사항】",  # 쟁점 부분 구분
            "\n【처리경위】",  # 처리 과정 구분
            "\n【처리결과】",  # 결과 부분 구분
            "\n■", "\n\n", "\n", ".", " ", ""
        ]
    )
    docs = text_splitter.split_documents(documents)
    print(f"PDF 문서에서 분할된 청크의 수: {len(docs)}개")
    if docs:
        print("첫 번째 청크 내용 (일부):\n", docs[0].page_content[:300], "...") # 첫 300자만 출력
    else:
        print("문서가 성공적으로 분할되지 않았습니다. PDF 파일 내용이 비어있거나 형식에 문제가 있을 수 있습니다.")
else:
    docs = []
    print("로드된 문서가 없어 문서 분할을 건너뜁니다.")

'../data/콘텐츠분쟁해결_사례.pdf'에서 109 페이지의 문서 로드 완료.
PDF 문서에서 분할된 청크의 수: 104개
첫 번째 청크 내용 (일부):
 콘텐츠분쟁조정 법리 연구 2부- 타 분쟁조정사례 조사 - ...


In [3]:
# --- 2단계: 임베딩 모델 설정 ---
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 한국어 법률 용어에 최적
    dimensions=1536  # 고성능 임베딩 차원
)

# --- 3단계: 벡터 저장소 및 검색기 설정 ---
# FAISS를 사용하여 인메모리 벡터 저장소를 생성합니다.
print("문서 임베딩 및 벡터 저장소 생성 중... (시간이 다소 소요될 수 있습니다)")
if docs: # 분할된 문서가 있을 때만 실행
    vectorstore = FAISS.from_documents(docs, embeddings)
    print("벡터 저장소 생성 완료.")

    retriever = vectorstore.as_retriever(
        search_type="similarity",  # "mmr" (다양성 고려) 또는 "similarity" (유사도만 고려)
        search_kwargs={"k": 5}  # 상위 5개 관련 사례 검색
    )
    print(f"검색기 설정 완료 (상위 {retriever.search_kwargs['k']}개 문서 검색).")
else:
    vectorstore = None
    retriever = None
    print("분할된 문서가 없어 벡터 저장소 및 검색기를 생성할 수 없습니다. 이전 단계를 확인해주세요.")

문서 임베딩 및 벡터 저장소 생성 중... (시간이 다소 소요될 수 있습니다)
벡터 저장소 생성 완료.
검색기 설정 완료 (상위 5개 문서 검색).


In [4]:
# --- 4단계: LLM 설정 ---
llm = ChatOpenAI(
    model="gpt-4o",  # 최신 또는 "gpt-4o-mini" (경제적 대안)
    temperature=0.2,  # 법률 조언은 정확성 중시 (0에 가까울수록 일관적, 1에 가까울수록 창의적)
    max_tokens=2000  # 충분한 답변 길이 확보
)
print(f"LLM 설정 완료: {llm.model_name}, Temperature: {llm.temperature}")

# --- 5단계: 법률 자문 프롬프트 작성 ---
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다.
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요            # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요         # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요   # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요          # 실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요 # 한계 인정

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print("법률 자문 프롬프트 템플릿 작성 완료.")

LLM 설정 완료: gpt-4o, Temperature: 0.2
법률 자문 프롬프트 템플릿 작성 완료.


In [5]:
# --- 6단계: QA 체인 생성 ---
qa_chain = None # qa_chain 초기화
if retriever: # 검색기가 성공적으로 설정되었을 때만 QA 체인 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,  # 위에서 설정한 LLM
        chain_type="stuff",  # 검색된 모든 문서를 하나의 프롬프트로 합쳐서 LLM에 전달
        retriever=retriever,  # 위에서 설정한 검색기
        chain_type_kwargs={"prompt": prompt},  # 정의한 법률 자문 프롬프트
        return_source_documents=True  # 참조 문서도 함께 반환
    )
    print("QA 체인 생성 완료.")
else:
    print("검색기가 설정되지 않아 QA 체인을 생성할 수 없습니다. 이전 단계를 확인해주세요.")


# --- 7단계: 테스트 질문 작성 및 실행 ---
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

if qa_chain: # QA 체인이 성공적으로 생성되었을 때만 테스트 실행
    print("\n--- RAG 시스템 테스트 시작 ---")
    for i, question in enumerate(test_questions):
        print(f"\n[질문 {i+1}] {question}")
        try:
            result = qa_chain.invoke({"query": question}) # LangChain 0.2.0 부터는 .invoke() 사용
            print("\n[전문 법률 조언]")
            print(result["result"])
            print("\n[참조 문서 출처]")
            for doc in result["source_documents"]:
                # PDF 로더는 page_content와 metadata를 가집니다.
                print(f"- {doc.metadata.get('source', 'Unknown Source')}, Page: {doc.metadata.get('page', 'N/A')}")
            print("-" * 50)
        except Exception as e:
            print(f"질문 처리 중 오류 발생: {e}")
            print("-" * 50)
else:
    print("\nQA 체인이 준비되지 않아 테스트 질문을 실행할 수 없습니다.")

QA 체인 생성 완료.

--- RAG 시스템 테스트 시작 ---

[질문 1] 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

[전문 법률 조언]
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다.

1. **사례 기반 답변**:
   - 제시된 사례 1-가-1) 2006년 사례에서는 시스템 오류로 인해 아이템이 소멸된 경우, 계정 명의자가 아니면 복구 처리가 불가하다는 점이 강조되었습니다. 계정 명의자가 직접 복구 신청을 해야 한다는 점이 중요합니다.
   - 2009년 사례에서는 시스템 오류로 인한 손실이 주장되었으나, 당시 유사한 불만이 접수되지 않았고, 사용 내역상 문제가 없었기 때문에 시스템 오류로 보기 어렵다는 판단이 있었습니다.

2. **법적 근거 제시**:
   - 게임 서비스 이용약관에 따라 계정 명의자만이 복구 요청을 할 수 있다는 규정이 있을 수 있습니다. 따라서 이용약관을 확인하여 해당 규정이 있는지 검토해야 합니다.

3. **판례 참조**:
   - 2009년 사례에서 시스템 오류로 인한 손실이 인정되지 않은 경우를 참고하여, 시스템 오류가 실제로 발생했는지에 대한 증거가 중요합니다. 다른 사용자들에게 유사한 문제가 발생했는지 확인하는 것이 필요합니다.

4. **실무 가이드**:
   - **1단계**: 게임 서비스의 이용약관을 확인하여 계정 명의자와 관련된 규정을 검토합니다.
   - **2단계**: 시스템 오류가 발생했음을 입증할 수 있는 증거를 수집합니다. 예를 들어, 다른 사용자들도 동일한 문제를 겪었는지 확인합니다.
   - **3단계**: 게임 회사에 공식적으로 이의를 제기하고, 계정 명의자가 직접 복구 요청을 할 수 있도록 합니다.
   - **4단계**: 한국소비자원이나 관련 기관에 중재를 요청하여 문제 해결을 시도합니다.

5. **한계 인정**:
   - 제시된 사례집에서

In [6]:
# 8단계: 분쟁 유형 분류 함수 (선택 사항)
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

print("\n--- 분쟁 유형 분류 테스트 ---")
for question in test_questions:
    dispute_type = classify_dispute_type(question)
    print(f"질문: '{question}' -> 분류: {dispute_type}")


--- 분쟁 유형 분류 테스트 ---
질문: '온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?' -> 분류: 게임
질문: '인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?' -> 분류: 이러닝
질문: '무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?' -> 분류: 이러닝
질문: '미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?' -> 분류: 게임
질문: '온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?' -> 분류: 기타
